In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail_3'),
    #spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

2023-04-16 21:17:33.992 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0013:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230416-2117-0.2.109-b71b065e4bb6.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

### select matrices for skat:
1. sportsmen vs GTS 40
2. polish zeros vs GTS 40
3. polish no zeros vs GTS 40
1. 1kg  vs GTS 40

In [4]:
mt_1 = hl.read_matrix_table(localfs_path+'polesunion.mt')

In [5]:
mt_2 = hl.read_matrix_table(localfs_path+'poles_zerosunion.mt')

In [6]:
mt_1.count() # this is for polish controls only

(3928354, 1788)

In [7]:
mt_2.count()

(10733526, 1788)

In [8]:
pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)

2023-04-16 21:23:11.803 Hail: INFO: wrote table with 187 rows in 1 partition to /localfs/2307051/tmp_hail_3/persist_tableuoXo9kLr41
2023-04-16 21:23:12.970 Hail: INFO: Reading table to impute column types
2023-04-16 21:23:14.010 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading field 'kinship' as type str (imputed)
  Loading field 'disease' as type str (imputed)
  Loading field 'phenotype' as type str (imputed)
  Loading field 'add_pheno' as type str (imputed)
  Loading field 'heavy_tics' as type str (imputed)
  Loading field 'heavy_tics_familial' as type str (imputed)
  Loading field 'GTS_ASD_group' as type str (imputed)
  Loading field 'nonCTD' as type str (imputed)


In [9]:
mts = [mt_1, mt_2]

for idx, mt in enumerate(mts):
    mt = mt.annotate_cols(
    group = hl.if_else(
            mt.s.contains('B'),
            'local_controls',
            hl.if_else(
                (mt.s.contains('NA') | mt.s.contains('HG')),
                '1kg_controls',
                hl.if_else(
                    mt.s.contains('polish'),
                    'polish_controls',
                    'GTS'
                )
            )
        )
    )
    
    mt = mt.annotate_cols(phenotypes = pheno[mt.s])
    mts[idx] = mt

In [ ]:
#sportsmen vs GTS 40
#s_vs_gts = mts[1].filter_cols((mts[1].group == 'local_controls') | (mts[1].phenotypes.family == '.'))
#s_vs_gts = s_vs_gts.filter_rows(hl.agg.any(s_vs_gts.GT.is_non_ref()))
#s_vs_gts = s_vs_gts.checkpoint(localfs_path+'s_vs_gts.mt')    

# polish no zeros vs GTS 40
#p0_vs_gts = mts[0].filter_cols((mts[0].group == 'polish_controls') | (mts[0].phenotypes.family == '.'))
#p0_vs_gts = p0_vs_gts.filter_rows(hl.agg.any(p0_vs_gts.GT.is_non_ref()))
#p0_vs_gts = p0_vs_gts.checkpoint(localfs_path+'p0_vs_gts.mt') 

# polish zeros vs GTS 40
pw_vs_gts = mts[1].filter_cols((mts[1].group == 'polish_controls') | (mts[1].phenotypes.family == '.'))
pw_vs_gts = pw_vs_gts.filter_rows(hl.agg.any(pw_vs_gts.GT.is_non_ref()))
pw_vs_gts = pw_vs_gts.checkpoint(localfs_path+'pw_vs_gts.mt') 

# 1kg vs GTS 40
eur_vs_gts = mts[1].filter_cols((mts[1].group == '1kg_controls') | (mts[1].phenotypes.family == '.'))
eur_vs_gts = eur_vs_gts.filter_rows(hl.agg.any(eur_vs_gts.GT.is_non_ref()))
eur_vs_gts = eur_vs_gts.checkpoint(localfs_path+'eur_vs_gts.mt') 

In [ ]:
def run_pca(mtx, mtx_path):
    for_pca = mtx.filter_rows(mtx.variant_qc.AF[1] > 0.05)
    pruned_variant_table = hl.ld_prune(for_pca.GT, r2=0.2, bp_window_size=500000)
    for_pca = for_pca.filter_rows(hl.is_defined(pruned_variant_table[for_pca.row_key]))

    for_pca = for_pca.checkpoint(localfs_path+'for_pca_20_'+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'for_pca_'+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_20_'+mtx_path)
    
    return(mtx)

In [ ]:
#s_vs_gts = hl.read_matrix_table(localfs_path+'s_vs_gts.mt')    
#p0_vs_gts = hl.read_matrix_table(localfs_path+'p0_vs_gts.mt') 
pw_vs_gts = hl.read_matrix_table(localfs_path+'pw_vs_gts.mt') 
eur_vs_gts = hl.read_matrix_table(localfs_path+'eur_vs_gts.mt') 

In [ ]:
def run_pca_no_filter(mtx, mtx_path):
    
    for_pca = mtx.sample_rows(0.1)

    for_pca = for_pca.checkpoint(localfs_path+'subset_'+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'subset_'+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_no_filters_'+mtx_path)
    
    return(mtx)

In [ ]:
mts = [pw_vs_gts, eur_vs_gts]
mts_paths = ['pw_vs_gts', 'eur_vs_gts']
mts_2 = [pw_vs_gts, eur_vs_gts]

In [ ]:
for idx, mt in enumerate(mts):
    mts[idx] = run_pca(mt, mts_paths[idx])

In [ ]:
for idx, mt in enumerate(mts):
    mts_2[idx] = run_pca_no_filter(mt, mts_paths[idx])

In [ ]:
# WE STOPPED HETE

In [ ]:
# the european group for some reason did not compute, so I am recomputing it again
# this matrix has many more variant so I will subset it smaller

eur_vs_gts = hl.read_matrix_table(localfs_path+'eur_vs_gts.mt') 

for_pca = eur_vs_gts.sample_rows(0.05)

for_pca = for_pca.checkpoint(localfs_path+'subset_eur_vs_gts')
for_pca = hl.read_matrix_table(localfs_path+'subset_eur_vs_gts')
eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
eur_vs_gts = eur_vs_gts.annotate_cols(scores = pcs[eur_vs_gts.s].scores)
eur_vs_gts = eur_vs_gts.checkpoint(localfs_path+'after_pca_no_filters_eur_vs_gts')

2023-04-14 14:51:11.806 Hail: INFO: wrote matrix table with 424331 rows and 541 columns in 1995 partitions to /localfs/2215677/subset_eur_vs_gts
    Total size: 2.80 GiB
    * Rows/entries: 2.80 GiB
    * Columns: 3.38 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  1034 rows (7.37 MiB)
2023-04-14 14:52:41.929 Hail: INFO: hwe_normalize: found 422774 variants after filtering out monomorphic sites.
2023-04-14 14:52:45.832 Hail: INFO: pca: running PCA with 20 components... 1995]
2023-04-14 14:54:38.284 Hail: INFO: wrote table with 0 rows in 0 partitions to /localfs/2215677/tmp_hail_2/persist_tablejJclfNbAcV
    Total size: 87.92 KiB
    * Rows: 0.00 B
    * Globals: 87.92 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A


In [ ]:
# at this point we have a two different PCA options - mts
# PCA with LD prunning and without LD prunning -mts_2

In [40]:
gtf = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/resources/genecode_43/gencode.v43.basic.annotation.gtf',
    delimiter = "\t",
    no_header = True,
    comment = "#"
)

# filter the gtf to only contain protein-coding genes with any GO term
gtf = gtf.filter((gtf.f2 == 'gene') & (gtf.f8.contains('protein_coding')))

gtf = gtf.select(
    gene_position = hl.locus_interval(
        gtf.f0,
        hl.int(gtf.f3),
        hl.int(gtf.f4),
        reference_genome='GRCh38'
    ),
    gene_symbol = gtf.f8.split("\"")[5]
)  

go_genes = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/resources/human-genes-with-GO-and-symbols'
)

go_genes = go_genes.key_by(go_genes['UniProtKB Gene Name symbol'])

gtf = gtf.key_by(gtf['gene_symbol'])
gtf = gtf.filter(
    hl.is_defined(
        go_genes[gtf.gene_symbol]
    )
)

# overall we are keeping 18303 genes. At this poin the within gene intervals are extended by 20kb each side
start = hl.if_else(
    gtf.gene_position.start.position <= 20000, 1, gtf.gene_position.start.position - 20000
)

contig_len = hl.contig_length(gtf.gene_position.start.contig, reference_genome='GRCh38') 

stop = hl.if_else(
    (contig_len - gtf.gene_position.end.position) <= 20000,
    contig_len,
    gtf.gene_position.end.position + 20000
)

gtf = gtf.annotate(
    interval_20kb =
    hl.locus_interval(
        gtf.gene_position.start.contig,
        start,
        stop,
        reference_genome='GRCh38'
    )
)

gtf = gtf.key_by(gtf.interval_20kb)

2023-04-15 18:36:02.837 Hail: INFO: wrote table with 286313 rows in 1 partition to /localfs/2215677/tmp_hail_3/persist_tableBE4iqPoUQt
2023-04-15 18:36:03.448 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type str (not specified)
  Loading field 'f4' as type str (not specified)
  Loading field 'f5' as type str (not specified)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type str (not specified)
  Loading field 'f8' as type str (not specified)
2023-04-15 18:36:04.224 Hail: INFO: wrote table with 23805 rows in 1 partition to /localfs/2215677/tmp_hail_3/persist_tableDQUDpXtqZV
2023-04-15 18:36:04.700 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)


In [36]:
mts = [s_vs_gts, p0_vs_gts, pw_vs_gts, eur_vs_gts]
mts_2 = [s_vs_gts, p0_vs_gts, pw_vs_gts, eur_vs_gts]
mts_paths = ['s_vs_gts', 'p0_vs_gts', 'pw_vs_gts', 'eur_vs_gts']

In [37]:
for idx, mt in enumerate(mts):
    mts[idx] = hl.read_matrix_table(localfs_path+'after_pca_'+mts_paths[idx])
    mts_2[idx] = hl.read_matrix_table(localfs_path+'after_pca_no_filters_'+mts_paths[idx])

In [ ]:
# now annotate the standard PCA with filters with the other PCA (no filters)
# this is still lacking for the eur samples

for idx, mt in enumerate(mts[2:]): 
    mt = mt.annotate_rows(
        nearest_genes_20kb = hl.array(
            hl.set(
                gtf.index(mt.locus, all_matches=True)['gene_symbol']))
    )
    
    mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))
    mt = mt.explode_rows(mt.nearest_genes_20kb)
    
    mt_2 = hl.read_matrix_table(localfs_path+'after_pca_no_filters_'+mts_paths[2:][idx])

    mt = mt.annotate_cols(scores_no_filter = mt_2.cols()[mt.s].scores)
    
    mt = mt.checkpoint(localfs_path+'for_full_skat_'+mts_paths[2:][idx])
    mts[2:][idx] = mt

2023-04-15 18:36:05.165 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2023-04-15 18:36:09.408 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-04-15 18:36:09.929 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-04-15 18:36:13.612 Hail: INFO: Ordering unsorted dataset with network shuffle


In [ ]:
#repartition the full skat's

s_vs_gts = hl.read_matrix_table(
    '/localfs/2215677/for_full_skat_s_vs_gts'
)

s_vs_gts.naive_coalesce(1000).write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/s_vs_gts.mt'
)


In [ ]:
#repartition the full skat's

p0_vs_gts = hl.read_matrix_table(
    '/localfs/2215677/for_full_skat_p0_vs_gts'
)

p0_vs_gts.naive_coalesce(1000).write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/p0_vs_gts.mt'
)


In [4]:
# at this point I have two matrix-tables read:
s_vs_gts = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/s_vs_gts.mt'
)

p0_vs_gts = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/p0_vs_gts.mt'
)

##### annotate sportsmen and 1kg samples with sex

In [ ]:
F = ['B434','B437','B440','B447','B514','B519','B521']
mts = [s_vs_gts, p0_vs_gts, pw_vs_gts, eur_vs_gts]
mts_paths = ['s_vs_gts', 'p0_vs_gts', 'pw_vs_gts', 'eur_vs_gts']

In [ ]:
#at this point run some QC to compare the groups (overall) - do this later

In [ ]:
for mt in mts:
    

In [60]:
# add sex as covariate !!!!


def run_skat_log(mtx, pcs):
    
    scores = [mtx.scores[x] for x in list(range(pcs))]
    
    skat_table = hl.skat(key_expr=mtx.nearest_genes_20kb,
                         weight_expr=mtx.cadd.score_phred,
                         y=(mtx.group == 'GTS'),
                         x=mtx.GT.n_alt_alleles(),
                         covariates=[1] + scores,
                         max_size = 2500,
                         logistic = True)

    skat_table.order_by(skat_table.p_value).show(20)
    
    qq_plot = hl.plot.qq(
        skat_table.p_value
    )
    show(qq_plot)
    
    return(skat_table, qq_plot)

In [65]:
skat_table_1, qq_plot_1 = run_skat_log(mts[0], 4)

2023-04-13 15:54:37.616 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-04-13 15:54:39.050 Hail: INFO: wrote table with 163 rows in 163 partitions to /localfs/2215677/tmp_hail/persist_tablei1fueEuThd
    Total size: 12.40 KiB
    * Rows: 12.39 KiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (55.00 B)
    * Largest partition:  1 rows (88.00 B)
2023-04-13 15:54:39.204 Hail: INFO: Coerced sorted dataset
2023-04-13 15:54:39.220 Hail: INFO: Coerced dataset with out-of-order partitions.


,,,,
id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CDHR5""",8,2.03e+02,9.80e-04,0
"""SCT""",10,2.03e+02,9.80e-04,0
"""DEAF1""",17,2.04e+02,1.06e-03,0
"""DRD4""",17,2.04e+02,1.06e-03,0
"""ANKK1""",137,1.76e+03,1.71e-03,0
"""DRD2""",356,3.47e+03,2.08e-03,0
"""TTC12""",54,4.84e+02,1.20e-02,0
"""DNAJC12""",23,6.99e+02,2.07e-02,0


2023-04-13 15:54:39.915 Hail: INFO: Coerced sorted dataset
2023-04-13 15:54:39.916 Hail: INFO: Coerced dataset with out-of-order partitions.
2023-04-13 15:54:41.038 Hail: INFO: wrote table with 163 rows in 150 partitions to /localfs/2215677/tmp_hail/persist_tableB6OzGPyXcU
2023-04-13 15:54:42.412 Hail: INFO: wrote table with 163 rows in 150 partitions to /localfs/2215677/tmp_hail/persist_tableI7AhVWz8T7


In [74]:
skat_table_2, qq_plot_2 = run_skat_log(mts[2], 5)

FatalError: HailException: Failed to fit logistic regression null model (MLE with covariates only): Newton iteration failed to converge

Java stack trace:
is.hail.utils.HailException: Failed to fit logistic regression null model (MLE with covariates only): Newton iteration failed to converge
	at is.hail.utils.ErrorHandling.fatal(ErrorHandling.scala:17)
	at is.hail.utils.ErrorHandling.fatal$(ErrorHandling.scala:17)
	at is.hail.utils.package$.fatal(package.scala:78)
	at is.hail.methods.Skat.logisticSkat$1(Skat.scala:263)
	at is.hail.methods.Skat.execute(Skat.scala:308)
	at is.hail.expr.ir.functions.WrappedMatrixToTableFunction.execute(RelationalFunctions.scala:52)
	at is.hail.expr.ir.TableToTableApply.execute(TableIR.scala:3363)
	at is.hail.expr.ir.TableIR.analyzeAndExecute(TableIR.scala:61)
	at is.hail.expr.ir.Interpret$.run(Interpret.scala:864)
	at is.hail.expr.ir.Interpret$.alreadyLowered(Interpret.scala:58)
	at is.hail.expr.ir.LowerOrInterpretNonCompilable$.evaluate$1(LowerOrInterpre

In [ ]:
samples_1 = mts[1].s.collect()

In [ ]:
mt_5 = mts[1].filter_cols(hl.literal(samples_1[0:80]).contains(mts[1].s))

In [ ]:
skat_table_5, genes_result_5, qq_plot_5 = run_skat_log(mt_5, gene_list, 4)

In [ ]:
skat_table_6, genes_result_6, qq_plot_6 = run_skat_log(mt_5, gene_list, 4)

In [ ]:
samples_2 = mts[3].s.collect()

In [ ]:
samples_2[-80:]

In [ ]:
mt_6 = mts[3].filter_cols(hl.literal(samples_2[-80:]).contains(mts[3].s))

In [ ]:
skat_table_7, genes_result_7, qq_plot_7 = run_skat_log(mt_6, gene_list, 4)

## Now run SKAT on selected genes only

In [ ]:
gene_list = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2', 'NR2F1', 'ASH1L', 'FLT3']

In [ ]:
#for idx, mt in enumerate(mts): 
#    mt = mt.annotate_rows(
#        nearest_genes_20kb = hl.array(
#            hl.set(
#                gtf.index(mt.locus, all_matches=True)['gene_symbol']))
#    )
    
#    mt = mt.filter_rows(hl.any(lambda x: hl.literal(gene_list).contains(x), mt.nearest_genes_20kb))
#    mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))
#    mt = mt.explode_rows(mt.nearest_genes_20kb)
#    mt = mt.filter_rows(hl.literal(gene_list).contains(mt.nearest_genes_20kb))
#    mt = mt.checkpoint(localfs_path+'for_skat_'+mts_paths[idx])
#    mts[idx] = mt